# Exploratory Analysis

## Setup

In [1]:
IN_COLAB = False
print("Running as a Jupyter notebook - intended for development only!")
from IPython import get_ipython

ipython = get_ipython()
# Code to automatically update the EasyTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_15025/3477142988.py:7: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_15025/3477142988.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import ast
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader
import pandas as pd
from IPython.display import clear_output

from torchtyping import TensorType as TT
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
from torchtyping import TensorType

In [3]:
import circuitsvis
from circuitsvis import attention

import easy_transformer
import easy_transformer.utils as utils
from easy_transformer.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from easy_transformer import EasyTransformer, EasyTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [4]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [5]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Prompts to analyse

### Model setup

We're mainly interested in the SoLU model, but can also use the attn model for
comparison.

In [6]:
model = EasyTransformer.from_pretrained(
    "NeelNanda/SoLU_1L512W_C4_Code",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

attn_model = EasyTransformer.from_pretrained(
    "NeelNanda/Attn_Only_1L512W_C4_Code",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    )

Loading model: NeelNanda/SoLU_1L512W_C4_Code
Moving model to device:  cuda
Finished loading pretrained model NeelNanda/SoLU_1L512W_C4_Code into EasyTransformer!
Loading model: NeelNanda/Attn_Only_1L512W_C4_Code
Moving model to device:  cuda
Finished loading pretrained model NeelNanda/Attn_Only_1L512W_C4_Code into EasyTransformer!


### Verify the model gets it right

In [7]:
# Using a standard format
example_prompt = "There was a notable person called 'Stephen Haw"
example_answer = "king"
prompt = example_prompt + example_answer
utils.test_prompt(example_prompt, example_answer, model=model, prepend_bos=True, prepend_space_to_answer=False)

Tokenized prompt: ['<|BOS|>', 'There', ' was', ' a', ' notable', ' person', ' called', " '", 'Stephen', ' Haw']
Tokenized answer: ['king']


Performance on answer token:
Rank: 0        Logit: 20.73 Prob: 56.08% Token: |king|

Top 0th token. Logit: 20.73 Prob: 56.08% Token: |king|
Top 1th token. Logit: 19.26 Prob: 12.92% Token: |th|
Top 2th token. Logit: 18.47 Prob:  5.86% Token: |kes|
Top 3th token. Logit: 18.35 Prob:  5.22% Token: |ker|
Top 4th token. Logit: 18.21 Prob:  4.54% Token: |ley|
Top 5th token. Logit: 17.57 Prob:  2.39% Token: |orth|
Top 6th token. Logit: 17.53 Prob:  2.29% Token: |ken|
Top 7th token. Logit: 17.05 Prob:  1.41% Token: |key|
Top 8th token. Logit: 16.96 Prob:  1.30% Token: |ke|
Top 9th token. Logit: 16.68 Prob:  0.98% Token: |son|


Ranks of the answer tokens: [('king', 0)]

### Generate a bunch of prompts and answers

In [ ]:
data = pd.read_csv("data/sample_names.csv")

prompts = []
answers = []
model_results = []

# Iterate over pandas rows
for index, row in data.iterrows():
    tokens = ast.literal_eval(row["tokenized"])
    bigram = row["bigrams"]
    
    prompt_str = model.tokenizer.decode(tokens[0:2])
    answer_str = model.tokenizer.decode(tokens[2])
    bigram_str = model.tokenizer.decode(bigram)
    
    print(model(prompt_str).shape)
    break
    res = torch.argmax(model(prompt_str), dim=-1).item()
    model_results.append(res)
    
    prompts.append("There was a notable person called '"+ prompt_str)
    answers.append((answer_str, bigram_str))

# clear_output(wait=True)
 
# pd.DataFrame({"prompt":prompts, "answer":answers, "model_res": model_results}).head(10)

In [ ]:
# Note prompts need to be the same length (in tokens) for activation patching
# (whereas they don't appear to need this for the other methods)
# TODO: Think about making strings by hand (by token numbers)
# prompt_format = [
#     # ".he{}odes .he{}odes .he",
#     # TODO: Consider replacing self/odes stuff with a lot of random things
#     "self.he{}odes self.he{}odes self.he",
#     "this.he{}odes this.he{}odes this.he", # TODO: re-run with this fixed
#     # ".he{} .he{} .he",
# ]

# names = [
#     ("xc", "xa"),
#     ("xc", "xa"),
#     # ("xc", "xa"),
# ]

# # List of prompts
# prompts = []
# # List of answers, in the format (correct, incorrect)
# answers = []

# for i in range(len(prompt_format)):
#     for j in range(2):
#         correct = names[i][j]
#         incorrect = names[i][1 - j]
#         prompts.append(prompt_format[i].format(correct, correct))
#         answers.append((correct, incorrect))

# print(prompts)
# print(answers)

for prompt in prompts:
    assert len(model.to_tokens(prompt)) == len(model.to_tokens(prompts[0]))

# Answer tokens = [(correct_token, incorrect_token)... for each prompt]
answer_tokens = [(model.to_single_token(answer[0]), model.to_single_token(answer[1])) for answer in answers]
answer_tokens = torch.tensor(answer_tokens).cuda()
print(answer_tokens)

### Run with the cache

In [ ]:
# Get the tokens for each prompt [ prompt x tokens ]
# Note shorter prompts will be padded
tokens = model.to_tokens(prompts, prepend_bos=True)
tokens = tokens.cuda()
print(tokens.shape)

# Run the model and cache all activations
original_logits, cache = model.run_with_cache(tokens)

# Logits [ prompt x token x logits]
original_logits.shape

### Setup logit diff function

We'll later be evaluating how model performance differs upon performing various
interventions, so it's useful to have a metric to measure model performance. Our
metric here will be the **logit difference**, the difference in logit between
the correct and incorrect answers.

In [ ]:
def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
    if per_prompt:
        return answer_logit_diff
    else:
        return answer_logit_diff.mean()

print("Per prompt logit difference:", logits_to_ave_logit_diff(original_logits, answer_tokens, per_prompt=True))
original_average_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
print("Average logit difference:", logits_to_ave_logit_diff(original_logits, answer_tokens).item())

$e^{4.3}\approx 75\times$ higher probability on the correct answer. 

## Brainstorm what's going on

In general I think that commonly "he" will be followed by "xa" for e.g.
"hexadecimal", so this is probably the default bigram solution. A single
attention layer can't do better than this as it can't do skip trigrams (i.e. he
-> he -> xc). However adding an MLP can potentially do this, if for some reason
the xc token is attended to by one head (and then the MLP layer can adjudicate
between this and other potential next tokens).

One way to have this is a "repeated bigrams head" where destination tokens such
as "he" will attend to source tokens such as "xa" (e.g. for hexadecimal) and
"xc" (e.g. for "hexcodes"). If this is the case, we'd expect similar things
where bigram statistics give two close output tokens.

## Direct Logit Attribution

We'll compare the correct and incorrect output tokens using logit difference directions.

In [ ]:
# Answer residual directions [ prompt x correct/incorrect (i.e. 2) x d_model ]
answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)
print("Answer residual directions shape:", answer_residual_directions.shape)

# Logit difference directions [ prompt x d_model ]
logit_diff_directions = answer_residual_directions[:, 0] - answer_residual_directions[:, 1]
print("Logit difference directions shape:", logit_diff_directions.shape)

To verify that this works, we can apply this to the final residual stream for our cached prompts (after applying LayerNorm scaling) and verify that we get the same answer. 

In [ ]:
# Scope to function to avoid conflicts
def verify_resid_directions():
    # cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type]. 
    final_residual_stream = cache["resid_post", -1]
    # print(dict(cache).keys())
    print("Final residual stream shape:", final_residual_stream.shape)
    final_token_residual_stream = final_residual_stream[:, -1, :]
    # Apply LayerNorm scaling
    # pos_slice is the subset of the positions we take - here the final token of each prompt
    scaled_final_token_residual_stream = cache.apply_ln_to_stack(final_token_residual_stream, layer = -1, pos_slice=-1)

    average_logit_diff = einsum("batch d_model, batch d_model -> ", scaled_final_token_residual_stream, logit_diff_directions)/len(prompts)
    print("Calculated average logit diff:", average_logit_diff.item())
    print("Original logit difference:",original_average_logit_diff.item())
    
verify_resid_directions()

### Logit Lens

In [ ]:
def residual_stack_to_logit_diff(
    residual_stack: TT["components", "prompt", "d_model"], 
    cache: ActivationCache,
    logit_difference_directions: TT["prompt", "d_model"],
    prompts_list: List[str]
    ) -> TT["components"]:
    """Per component (layer) residual values -> per component logit difference

    Args:
        residual_stack: Per layer residual (e.g. the residuals right after each layer)
        cache: Cached activations from a model run
        logit_difference_directions: Logit difference directions [ prompt x d_model ]
        prompts_list: List of prompts

    Returns:
        Logit difference per layer
    """
    # Get the residual stack (after each component/layer), scaled by the
    # LayerNorm after each layer.
    scaled_residual_stack: TT["components", "prompt", "d_model"] = cache.apply_ln_to_stack(
        residual_stack, 
        layer = -1, # Layer we're taking the input to (-1 = unembed)
        pos_slice=-1 # Position we're taking the input to (-1 = final token) 
        )
    
    sum_logit_diff_per_component: TT["components"] = einsum(
        "components prompt d_model, prompt d_model -> components", 
        scaled_residual_stack, 
        logit_difference_directions
        )
    
    # Divide by number of prompts. TODO: Why????
    return sum_logit_diff_per_component/len(prompts_list)


In [ ]:
# Get the (accumulated) residuals after each layer
per_layer_residual, labels = cache.decompose_resid(layer=-1, pos_slice=-1, return_labels=True)

# Get the logit difference per layer
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, cache, logit_diff_directions, prompts)

line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

# TODO: Replace first occurrence with he by something else -> use as baseline
# (rather than xc/xa). Also try xa in other parts of the text.

This shows broadly that the attention layer helps (which we'd expect if a head
is attending to common pairings), and then the MLP layer gets us over the line.

__TODO: A question I have is__ does the attention layer alone get it right (i.e. in
this model is the MLP layer needed). It's possible that the architecture, in
allowing for adjudication, actually also just gets things right without it
(whereas without the MLP layer it's not really incentivised to setup a "common
bigrams head").

## Head Attribution

In [ ]:
# Get a stack of all the head contributions to the residual stream
per_head_residual, labels = cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)

per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, cache, logit_diff_directions, prompts)
per_head_logit_diffs = einops.rearrange(per_head_logit_diffs, "(layer head_index) -> layer head_index", layer=model.cfg.n_layers, head_index=model.cfg.n_heads)
imshow(per_head_logit_diffs, labels={"x":"Head", "y":"Layer"}, title="Logit Difference From Each Head")

### Attention Analysis

In [ ]:
def visualize_attention_patterns(
    heads: Union[List[int], int, TT["heads"]], 
    model_number_heads: int,
    batch_item_index: int,
    local_cache: ActivationCache, 
    local_tokens: torch.Tensor,
    title: str=""):
    
    # Convert list of head numbers to numpy array
    if isinstance(heads, int):
        heads = [heads]
    if isinstance(heads, list) or isinstance(heads, torch.Tensor):
        heads = utils.to_numpy(heads)
    
    # Labels/patterns
    labels = []
    patterns = []
    
    for head in heads:
        layer = head // model_number_heads
        head_index = head % model_number_heads
        # Get the attention patterns for the head
        # Attention patterns have shape [batch, head_index, query_pos, key_pos]
        patterns.append(local_cache["attn", layer][batch_item_index, head_index])
        labels.append(f"L{layer}H{head_index}")
        
    str_tokens = model.to_str_tokens(local_tokens)
    patterns = torch.stack(patterns, dim=-1)
    patterns = einops.rearrange(patterns, "query_pos key_pos heads -> heads query_pos key_pos")
    
    # Plot
    attention_vis = attention.attention_heads(attention=patterns, tokens=str_tokens, attention_head_names=labels)
    
    # Display
    display(HTML(f"<h2>{title}</h2>"))
    display(attention_vis)

In [ ]:
# Get the top k head numbers (by highest positive logit difference)
top_k = 4
top_positive_logit_attr_heads = torch.topk(per_head_logit_diffs.flatten(), k=top_k).indices
print(top_positive_logit_attr_heads)

# Prompt to visualize
prompt_number = 0

# Visualize the attention patterns for them
visualize_attention_patterns(
    top_positive_logit_attr_heads,
    model_number_heads=model.cfg.n_heads,
    batch_item_index=prompt_number,
    local_cache=cache, 
    local_tokens=tokens[prompt_number],
    title=f"Top {top_k} Positive Logit Attribution Heads - Prompt {prompt_number}"
    )


top_negative_logit_attr_heads = torch.topk(-per_head_logit_diffs.flatten(), k=top_k).indices
visualize_attention_patterns(
    top_negative_logit_attr_heads,
    model_number_heads=model.cfg.n_heads,
    batch_item_index=prompt_number,
    local_cache=cache, 
    local_tokens=tokens[prompt_number],    
    title=f"Top {top_k} Negative Logit Attribution Heads")

Kind of interesting that H6 appears to be doing bigram style stuff. E.g. `self`
(dest) -> `.` (src), he -> xc, xc -> odes, dot-> xc

Also showing with the counter-factual ("xa" not "xc" as the correct answer), to
check it's still doing some sort of fake induction:

In [ ]:
# Prompt to visualize
prompt_number = 1

# Visualize the attention patterns for them
visualize_attention_patterns(
    top_positive_logit_attr_heads,
    model_number_heads=model.cfg.n_heads,
    batch_item_index=prompt_number,
    local_cache=cache, 
    local_tokens=tokens[prompt_number],
    title=f"Top {top_k} Positive Logit Attribution Heads - Prompt {prompt_number}"
    )

Seems like H6 is a repeated bigrams head (responds to both `xa` and `xc`).

## Activation Patching

### Setup

In [ ]:
# Alternative the prompt order to incorrect, correct... so that the answers are
# now all incorrect rather than all correct.
corrupted_prompts = []
for i in range(0, len(prompts), 2):
    corrupted_prompts.append(prompts[i+1])
    corrupted_prompts.append(prompts[i])
print(corrupted_prompts)

# Run the model with the corrupted tokens
corrupted_tokens = model.to_tokens(corrupted_prompts, prepend_bos=True)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, return_type="logits")

# Get the "corrupted" logit differences (which should be the opposite of the non-corrupted ones)
corrupted_average_logit_diff = logits_to_ave_logit_diff(corrupted_logits, answer_tokens)
print("Corrupted Average Logit Diff", corrupted_average_logit_diff)
print("Clean Average Logit Diff", original_average_logit_diff)

In [ ]:
model.to_string(corrupted_tokens)

We now intervene on the corrupted run and patch in the clean residual stream at a specific layer and position.

We do the intervention using EasyTransformer's `HookPoint` feature. We can design a hook function that takes in a specific activation and returns an edited copy, and temporarily add it in with `model.run_with_hooks`. 

In [ ]:
def patch_residual_component(
    corrupted_residual_component: TT["batch", "pos", "d_model"],
    hook, 
    pos, 
    clean_cache
    ):
    """Patch a corrupted residual component, with the clean residual component
    
    For a specific residual component, for all prompts, in a set position, all the residual values
    """
    corrupted_residual_component[:, pos, :] = clean_cache[hook.name][:, pos, :]
    return corrupted_residual_component

def normalize_patched_logit_diff(patched_logit_diff):
    """Normalize the patched logit difference (to be between -1 and 1)
    
    Subtract corrupted logit diff to measure the improvement, divide by the
    total improvement from clean to corrupted to normalize.
    
    0 means zero change, negative means actively made worse, 1 means totally
    recovered clean performance, >1 means actively *improved* on clean
    performance.
    """
    return (patched_logit_diff - corrupted_average_logit_diff)/(original_average_logit_diff - corrupted_average_logit_diff)

# Patched logit difference for each attention layer (just 1) and position (14)
# Initialize:
patched_residual_stream_diff: TT["attention_layers", "pos"] = torch.zeros(
    model.cfg.n_layers, 
    tokens.shape[1], 
    device="cuda", 
    dtype=torch.float32
    )

# For each attention layer (note there is really just one layer in this case!)
for layer in range(model.cfg.n_layers):
    # For each position in the prompt
    for position in range(tokens.shape[1]):
        # Run the model with patched residual components for just this layer &
        # token
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_logits: TT["batch", "pos", "vocab"] = model.run_with_hooks(
            corrupted_tokens, 
            fwd_hooks = [(utils.act_name("resid_pre", layer), 
                hook_fn)], 
            return_type="logits"
        )
        
        # Note this would be broken if the final token is not the last in the
        # prompt (due to end padding)
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_residual_stream_diff[layer, position] = normalize_patched_logit_diff(patched_logit_diff)
        


### Attention layers (not that useful as just 1)

In [ ]:
prompt_position_labels = [f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(tokens[0]))]
imshow(patched_residual_stream_diff, x=prompt_position_labels, title="Logit Difference From Patched Residual Stream", labels={"x":"Position", "y":"Layer"})

Patching the residual for the attention layer, at the xc/xa tokens (there are 2),
basically recovers 50% of performance for each one (i.e. 100% in total).

### MLP layers (not that useful as just 1)

We can apply exactly the same idea, but this time patching in attention or MLP layers. These are also residual components with identical shapes to the residual stream terms, so we can reuse the same hooks.

In [ ]:
patched_attn_diff = torch.zeros(model.cfg.n_layers, tokens.shape[1], device="cuda", dtype=torch.float32)
patched_mlp_diff = torch.zeros(model.cfg.n_layers, tokens.shape[1], device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for position in range(tokens.shape[1]):
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_attn_logits = model.run_with_hooks(
            corrupted_tokens, 
            fwd_hooks = [(utils.act_name("attn_out", layer), 
                hook_fn)], 
            return_type="logits"
        )
        patched_attn_logit_diff = logits_to_ave_logit_diff(patched_attn_logits, answer_tokens)
        patched_mlp_logits = model.run_with_hooks(
            corrupted_tokens, 
            fwd_hooks = [(utils.act_name("mlp_out", layer), 
                hook_fn)], 
            return_type="logits"
        )
        patched_mlp_logit_diff = logits_to_ave_logit_diff(patched_mlp_logits, answer_tokens)

        patched_attn_diff[layer, position] = normalize_patched_logit_diff(patched_attn_logit_diff)
        patched_mlp_diff[layer, position] = normalize_patched_logit_diff(patched_mlp_logit_diff)

In [ ]:
imshow(patched_attn_diff, x=prompt_position_labels, title="Logit Difference From Patched Attention Layer", labels={"x":"Position", "y":"Layer"})

The MLP layer already has the important info moved to the "he" token at the end,
and this is the only one that matters for the final prediction so of course this
will get 100%.

In [ ]:
imshow(patched_mlp_diff, x=prompt_position_labels, title="Logit Difference From Patched MLP Layer", labels={"x":"Position", "y":"Layer"})

### Heads (not that useful)

In [ ]:
# We can refine the above analysis by patching in individual heads! This is
# somewhat more annoying, because there are now three dimensions (head_index,
# position and layer), so for now lets patch in a head's output across all
# positions. 

# The easiest way to do this is to patch in the activation `z`, the "mixed
# value" of the attention head. That is, the average of all previous values
# weighted by the attention pattern, ie the activation that is then multiplied
# by `W_O`, the output weights.  

def patch_head_vector(
    corrupted_head_vector: TT["batch", "pos", "head_index", "d_head"],
    hook, 
    head_index, 
    clean_cache):
    corrupted_head_vector[:, :, head_index, :] = clean_cache[hook.name][:, :, head_index, :]
    return corrupted_head_vector


patched_head_z_diff = torch.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens, 
            fwd_hooks = [(utils.act_name("z", layer, "attn"), 
                hook_fn)], 
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_z_diff[layer, head_index] = normalize_patched_logit_diff(patched_logit_diff)

In [ ]:
imshow(patched_head_z_diff, title="Logit Difference From Patched Head Output", labels={"x":"Head", "y":"Layer"})

### Decomposing Heads

#### Value patching (OV Circuit)

In [ ]:
# First let's patch in the value vectors, to measure when figuring out what to
# move is important. . This has the same shape as z ([batch, pos, head_index,
# d_head]) so we can reuse the same hook. 

patched_head_v_diff = torch.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens, 
            fwd_hooks = [(utils.act_name("v", layer, "attn"), 
                hook_fn)], 
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_v_diff[layer, head_index] = normalize_patched_logit_diff(patched_logit_diff)

In [ ]:
imshow(patched_head_v_diff, title="Logit Difference From Patched Head Value", labels={"x":"Head", "y":"Layer"})

#### Value patching vs head output patching

In [ ]:
head_labels = [f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)]
scatter(
    x=utils.to_numpy(patched_head_v_diff.flatten()), 
    y=utils.to_numpy(patched_head_z_diff.flatten()), 
    xaxis="Value Patch",
    yaxis="Output Patch",
    caxis="Layer",
    hover_name = head_labels,
    color=einops.repeat(np.arange(model.cfg.n_layers), "layer -> (layer head)", head=model.cfg.n_heads),
    # range_x=(-0.5, 0.5),
    # range_y=(-0.5, 0.5),
    title="Scatter plot of output patching vs value patching")

My impression is that this means the OV circuit is vital (how to move things)
when looking at the difference between the corrupted and uncorrupted logits,
which makes sense as the attention would be the same between both potential
bigrams. 

#### Attention (QK) patching

In [ ]:
def patch_head_pattern(
    corrupted_head_pattern: TT["batch", "head_index", "query_pos", "d_head"],
    hook, 
    head_index, 
    clean_cache):
    corrupted_head_pattern[:, head_index, :, :] = clean_cache[hook.name][:, head_index, :, :]
    return corrupted_head_pattern

patched_head_attn_diff = torch.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=torch.float32)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_pattern, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens, 
            fwd_hooks = [(utils.act_name("attn", layer, "attn"), 
                hook_fn)], 
            return_type="logits"
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_attn_diff[layer, head_index] = normalize_patched_logit_diff(patched_logit_diff)


In [ ]:
imshow(patched_head_attn_diff, title="Logit Difference From Patched Head Pattern", labels={"x":"Head", "y":"Layer"})
head_labels = [f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)]
scatter(
    x=utils.to_numpy(patched_head_attn_diff.flatten()), 
    y=utils.to_numpy(patched_head_z_diff.flatten()), 
    hover_name = head_labels,
    xaxis="Attention Patch",
    yaxis="Output Patch",
    title="Scatter plot of output patching vs attention patching")

## Consolidating Understanding [TODO]

### TODO

- Look at specific MLP neurons (full_resid_decomp helps do direct logit
  attribution) <HIGH>
- Prove it is inductiony (e.g. remove he / make further apart e.g. 100s apart) <HIGHEST
- If this fails, look for other induction promps (search data for prompts that
  contain repeated bigrams, where 2nd occurance better than 1st & run same
  tests).
- Where unembed vectors have a high cosine similarity (e.g. all names), could
  look for neurons that have a high cosine similarity with the difference
  between these vectors

### Reference

#### Visualizing Attention Patterns

We can validate this by looking at the attention patterns of these heads! Let's take the top 10 heads by output patching (in absolute value) and split it into early, middle and late.

We see that middle heads attend from the final token to the second subject, and late heads attend from the final token to the indirect object, which is completely consistent with the above speculation! But weirdly, while *one* early head attends from the second subject to its first copy, the other two mysteriously attend to the word *after* the first copy.

In [ ]:
top_k = 10
top_heads_by_output_patch = torch.topk(patched_head_z_diff.abs().flatten(), k=top_k).indices
first_mid_layer = 7
first_late_layer = 9
early_heads = top_heads_by_output_patch[top_heads_by_output_patch<model.cfg.n_heads * first_mid_layer]
mid_heads = top_heads_by_output_patch[torch.logical_and(model.cfg.n_heads * first_mid_layer<=top_heads_by_output_patch, top_heads_by_output_patch<model.cfg.n_heads * first_late_layer)]
late_heads = top_heads_by_output_patch[model.cfg.n_heads * first_late_layer<=top_heads_by_output_patch]
visualize_attention_patterns(early_heads, title=f"Top Early Heads")
visualize_attention_patterns(mid_heads, title=f"Top Middle Heads")
visualize_attention_patterns(late_heads, title=f"Top Late Heads")

#### Comparing to the Paper

We can now refer to the (far, far more rigorous and detailed) analysis in the paper to compare our results! Here's the diagram they give of their results. 

![](https://pbs.twimg.com/media/FghGkTAWAAAmkhm.jpg)

(Head 1.2 in their notation is L1H2 in my notation etc. And note - in the [latest version of the paper](https://arxiv.org/pdf/2211.00593.pdf) they add 9.0 as a backup name mover, and remove 11.3)

The heads form three categories corresponding to the early, middle and late categhories we found and we did fairly well! Definitely not perfect, but with some fairly generic techniques and some a priori reasoning, we found the broad strokes of the circuit and what it looks like. We focused on the most important heads, so we didn't find all relevant heads in each category (especially not the heads in brackets, which are more minor), but this serves as a good base for doing more rigorous and involved analysis, especially for finding the *complete* circuit (ie all of the parts of the model which participate in this behaviour) rather than just a partial and suggestive circuit. Go check out [their paper](https://arxiv.org/abs/2211.00593) or [our interview](https://www.youtube.com/watch?v=gzwj0jWbvbo) to learn more about what they did and what they found!

Breaking down their categories:

* Early: The duplicate token heads, previous token heads and induction heads. These serve the purpose of detecting that the second subject is duplicated and which earlier name is the duplicate.
    * We found a direct duplicate token head which behaves exactly as expected, L3H0. Heads L5H0 and L6H9 are induction heads, which explains why they don't attend directly to the earlier copy of John!
    * Note that the duplicate token heads and induction heads do not compose with each other - both directly add to the S-Inhibition heads. The diagram is somewhat misleading.
* Middle: They call these S-Inhibition heads - they copy the information about the duplicate token from the second subject to the to token, and their output is used to *inhibit* the attention paid from the name movers to the first subject copy. We found all these heads, and had a decent guess for what they did.
    * In either case they attend to the second subject, so the patch that mattered was their value vectors!
* Late: They call these name movers, and we found some of them. They attend from the final token to the indirect object name and copy that to the logits, using the S-Inhibition heads to inhibit attention to the first copy of the subject token.
    * We did find their surprising result of *negative* name movers - name movers that inhibit the correct answer!
    * They have an entire category of heads we missed called backup name movers - we'll get to these later.

So, now, let's dig into the two anomalies we missed - induction heads and backup name mover heads

#### Early Heads are Induction Heads(?!)

A really weird observation is that some of the early heads detecting duplicated tokens are induction heads, not just direct duplicate token heads. This is very weird! What's up with that? 

First off, what's an induction head? An induction head is an important type of attention head that can detect and continue repeated sequences. It is the second head in a two head induction circuit, which looks for previous copies of the current token and attends to the token *after* it, and then copies that to the current position and predicts that it will come next. They're enough of a big deal that [we wrote a whole paper on them](https://transformer-circuits.pub/2022/in-context-learning-and-induction-heads/index.html).

![](https://pbs.twimg.com/media/FNWAzXjVEAEOGRe.jpg)

Second, why is it surprising that they come up here? It's surprising because it feels like overkill. The model doesn't care about *what* token comes after the first copy of the subject, just that it's duplicated. And it already has simpler duplicate token heads. My best guess is that it just already had induction heads around and that, in addition to their main function, they *also* only activate on duplicated tokens. So it was useful to repurpose this existing machinery. 

This suggests that as we look for circuits in larger models life may get more and more complicated, as components in simpler circuits get repurposed and built upon. 

We can verify that these are induction heads by running the model on repeated text and plotting the heads.

In [ ]:
example_text = "Research in mechanistic interpretability seeks to explain behaviors of machine learning models in terms of their internal components."
example_repeated_text = example_text + example_text
example_repeated_tokens = model.to_tokens(example_repeated_text, prepend_bos=True)
example_repeated_logits, example_repeated_cache = model.run_with_cache(example_repeated_tokens)
induction_head_labels = [81, 65]
visualize_attention_patterns(induction_head_labels, example_repeated_cache, example_repeated_tokens, title="Induction Heads")

One implication of this is that it's useful to categories heads according to whether they occur in simpler circuits, so that as we look for more complex circuits we can easily look for them. This is easy to do here! An interesting fact about induction heads is that they work on a sequence of repeated random tokens - notable for being wildly off distribution from the natural language GPT-2 was trained on. Being able to predict a model's behaviour off distribution is a good mark of success for mechanistic interpretability! This is a good sanity check for whether a head is an induction head or not. 

We can characterise an induction head by just giving a sequence of random tokens repeated once, and measuring the average attention paid from the second copy of a token to the token after the first copy. At the same time, we can also measure the average attention paid from the second copy of a token to the first copy of the token, which is the attention that the induction head would pay if it were a duplicate token head, and the average attention paid to the previous token to find previous token heads.

Note that this is a superficial study of whether something is an induction head - we totally ignore the question of whether it actually does boost the correct token or whether it composes with a single previous head and how. In particular, we sometimes get anti-induction heads which suppress the induction-y token (no clue why!), and this technique will find those too . But given the previous rigorous analysis, we can be pretty confident that this picks up on some true signal about induction heads.

<details> <summary>Technical Implementation Details</summary> 
We can do this again by using hooks, this time just to access the attention patterns rather than to intervene on them. 

Our hook function acts on the attention pattern activation. This has the name "blocks.{layer}.{layer_type}.hook_{activation_name}" in general, here it's "blocks.{layer}.attn.hook_attn". And it has shape [batch, head_index, query_pos, token_pos]. Our hook function takes in the attention pattern activation, calculates the score for the relevant type of head, and write it to an external cache.

We add in hooks using `model.run_with_hooks(tokens, fwd_hooks=[(names_filter, hook_fn)])` to temporarily add in the hooks and run the model, getting the resulting output. Previously names_filter was the name of the activation, but here it's a boolean function mapping activation names to whether we want to hook them or not. Here it's just whether the name ends with hook_attn. hook_fn must take in the two inputs activation (the activation tensor) and hook (the HookPoint object, which contains the name of the activation and some metadata such as the current layer).

Internally our hooks use the function `tensor.diagonal`, this takes the diagonal between two dimensions, and allows an arbitrary offset - offset by 1 to get previous tokens, seq_len to get duplicate tokens (the distance to earlier copies) and seq_len-1 to get induction heads (the distance to the token *after* earlier copies). Different offsets give a different length of output tensor, and we can now just average to get a score in [0, 1] for each head
</details>

In [ ]:
seq_len = 100
batch_size = 2

prev_token_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads), device="cuda")
def prev_token_hook(pattern, hook):
    layer = hook.layer()
    diagonal = pattern.diagonal(offset=1, dim1=-1, dim2=-2)
    # print(diagonal)
    # print(pattern)
    prev_token_scores[layer] = einops.reduce(diagonal, "batch head_index diagonal -> head_index", "mean")
duplicate_token_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads), device="cuda")
def duplicate_token_hook(pattern, hook):
    layer = hook.layer()
    diagonal = pattern.diagonal(offset=seq_len, dim1=-1, dim2=-2)
    duplicate_token_scores[layer] = einops.reduce(diagonal, "batch head_index diagonal -> head_index", "mean")
induction_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads), device="cuda")
def induction_hook(pattern, hook):
    layer = hook.layer()
    diagonal = pattern.diagonal(offset=seq_len-1, dim1=-1, dim2=-2)
    induction_scores[layer] = einops.reduce(diagonal, "batch head_index diagonal -> head_index", "mean")
original_tokens = torch.randint(100, 20000, size=(batch_size, seq_len))
repeated_tokens = einops.repeat(original_tokens, "batch seq_len -> batch (2 seq_len)").cuda()

pattern_filter = lambda act_name: act_name.endswith("hook_attn")
loss = model.run_with_hooks(repeated_tokens, return_type="loss", fwd_hooks=[(pattern_filter, prev_token_hook), (pattern_filter, duplicate_token_hook), (pattern_filter, induction_hook)])
print(utils.get_corner(prev_token_scores))
print(utils.get_corner(duplicate_token_scores))
print(utils.get_corner(induction_scores))

We can now plot the head scores, and instantly see that the relevant early heads are induction heads or duplicate token heads (though also that there's a lot of induction heads that are *not* use - I have no idea why!). 

In [ ]:

imshow(prev_token_scores, labels={"x":"Head", "y":"Layer"}, title="Previous Token Scores")
imshow(duplicate_token_scores, labels={"x":"Head", "y":"Layer"}, title="Duplicate Token Scores")
imshow(induction_scores, labels={"x":"Head", "y":"Layer"}, title="Induction Head Scores")

The above suggests that it would be a useful bit of infrastructure to have a "wiki" for the heads of a model, giving their scores according to some metrics re head functions, like the ones we've seen here. EasyTransformer makes this easy to make, as just changing the name input to `EasyTransformer.from_pretrained` gives a different model but in the same architecture, so the same code should work. If you want to make this, I'd love to see it! 

As a proof of concept, [I made a mosaic of all induction heads across the 40 models then in EasyTransformer](https://www.neelnanda.io/mosaic).

#### Backup Name Mover Heads

Another fascinating anomaly is that of the **backup name mover heads**. A standard technique to apply when interpreting model internals is ablations, or knock-out. If we run the model but intervene to set a specific head to zero, what happens? If the model is robust to this intervention, then naively we can be confident that the head is not doing anything important, and conversely if the model is much worse at the task this suggests that head was important. There are several conceptual flaws with this approach, making the evidence only suggestive, eg that the average output of the head may be far from zero and so the knockout may send it far from expected activations, breaking internals on *any* task. But it's still an easy technique to apply to give some data.

But a wild finding in the paper is that models have **built in redundancy**. If we knock out one of the name movers, then there are some backup name movers in later layers that *change their behaviour* and do (some of) the job of the original name mover head. This means that naive knock-out will significantly underestimate the importance of the name movers.


Let's test this! Let's ablate the most important name mover (head L9H9) on just the final token using a custom ablation hook and then cache all new activations and compared performance. We focus on the final position because we want to specifically ablate the direct logit effect. When we do this, we see that naively, removing the top name mover should reduce the logit diff massively, from 3.55 to 0.57. **But actually, it only goes down to 2.99!**

<details> <summary>Implementation Details</summary> 
Ablating heads is really easy in EasyTransformer! We can just define a hook on the z activation in the relevant attention layer (recall, z is the mixed values, and comes immediately before multiplying by the output weights $W_O$). z has a head_index axis, so we can set the component for the relevant head and for position -1 to zero, and return it. (Technically we could just edit in place without returning it, but by convention we always return an edited activation). 

We now want to compare all internal activations with a hook, which is hard to do with the nice `run_with_hooks` API. So we can directly access the hook on the z activation with `model.blocks[layer].attn.hook_z` and call its `add_hook` method. This adds in the hook to the *global state* of the model. We can now use run_with_cache, and don't need to care about the global state, because run_with_cache internally adds a bunch of caching hooks, and then removes all hooks after the run, *including* the previously added ablation hook. This can be disabled with the reset_hooks_end flag, but here it's useful! 
</details>

In [ ]:
top_name_mover = per_head_logit_diffs.flatten().argmax().item()
top_name_mover_layer = top_name_mover//model.cfg.n_heads
top_name_mover_head = top_name_mover % model.cfg.n_heads
print(f"Top Name Mover to ablate: L{top_name_mover_layer}H{top_name_mover_head}")
def ablate_top_head_hook(z: TT["batch", "pos", "head_index", "d_head"], hook):
    z[:, -1, top_name_mover_head, :] = 0
    return z
# Adds a hook into global model state
model.blocks[top_name_mover_layer].attn.hook_z.add_hook(ablate_top_head_hook)
# Runs the model, temporarily adds caching hooks and then removes *all* hooks after running, including the ablation hook.
ablated_logits, ablated_cache = model.run_with_cache(tokens)
print(f"Original logit diff: {original_average_logit_diff}")
print(f"Post ablation logit diff: {logits_to_ave_logit_diff(ablated_logits, answer_tokens).item()}")
print(f"Direct Logit Attribution of top name mover head: {per_head_logit_diffs.flatten()[top_name_mover].item()}")
print(f"Naive prediction of post ablation logit diff: {original_average_logit_diff - per_head_logit_diffs.flatten()[top_name_mover].item()}")

So what's up with this? As before, we can look at the direct logit attribution of each head to see what's going on. It's easiest to interpret if plotted as a scatter plot against the initial per head logit difference.

And we can see a *really* big difference in a few heads! (Hover to see labels) In particular the negative name mover L10H7 decreases its negative effect a lot, adding +1 to the logit diff, and the backup name mover L10H10 adjusts its effect to be more positive, adding +0.8 to the logit diff (with several other marginal changes). (And obviously the ablated head has gone down to zero!)

In [ ]:
per_head_ablated_residual, labels = ablated_cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
per_head_ablated_logit_diffs = residual_stack_to_logit_diff(per_head_ablated_residual, ablated_cache)
per_head_ablated_logit_diffs = per_head_ablated_logit_diffs.reshape(model.cfg.n_layers, model.cfg.n_heads)
imshow(per_head_ablated_logit_diffs, labels={"x":"Head", "y":"Layer"})
scatter(y=per_head_logit_diffs.flatten(), x=per_head_ablated_logit_diffs.flatten(), hover_name=head_labels, range_x=(-3, 3), range_y=(-3, 3), xaxis="Ablated", yaxis="Original", title="Original vs Post-Ablation Direct Logit Attribution of Heads")

One natural hypothesis is that this is because the final LayerNorm scaling has changed, which can scale up or down the final residual stream. This is slightly true, and we can see that the typical head is a bit off from the x=y line. But the average LN scaling ratio is 1.04, and this should uniformly change *all* heads by the same factor, so this can't be sufficient

In [ ]:
print("Average LN scaling ratio:", (cache["ln_final.hook_scale"][:, -1]/ablated_cache["ln_final.hook_scale"][:, -1]).mean().item())
print("Ablation LN scale", ablated_cache["ln_final.hook_scale"][:, -1])
print("Original LN scale", cache["ln_final.hook_scale"][:, -1])

**Exercise to the reader:** Can you finish off this analysis? What's going on here? Why are the backup name movers changing their behaviour? Why is one negative name mover becoming significantly less important?